In [ ]:
import numpy as np
import cv2
from skimage.feature import hog

def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2YUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    if conv == 'RGB2YUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features




In [ ]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob2
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
#from lesson_functions import *
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
%matplotlib qt


# Read in cars and notcars
cars = []
notcars = []
images = glob2.glob('./big_dataset_balanced/vehicles/**/*.png')
for image in images:
    cars.append(image)
images2 = glob2.glob('./big_dataset_balanced/non-vehicles/**/*.png')
for image in images2:
    notcars.append(image)

### TODO: Tweak these parameters and see how the results change.
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [None, None] # Min and max in y to search in slide_window()

car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

from sklearn.utils import shuffle

X = np.vstack((car_features, notcar_features)).astype(np.float64)    
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
print(y.shape)

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.15, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))



In [ ]:
# Use a linear SVC 
svc = LinearSVC(C= 1e-5,  random_state=40,  verbose=1) 
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

In [ ]:
## Save the model and other variables
import pickle
svc_pickle = {"svc": svc,
         "scaler": X_scaler,
         "orient": orient,
         "pix_per_cell": pix_per_cell,             
         "cell_per_block": cell_per_block, 
         "spatial_size": spatial_size,   
         "hist_bins":hist_bins }
pickle.dump(svc_pickle, open('svc_pickle_20160307_1.p', 'wb'))


In [ ]:
## Load the model and other variables
import pickle
dist_pickle = pickle.load( open("svc_pickle_20160307_1.p", "rb" ) )
svc = dist_pickle["svc"]
X_scaler = dist_pickle["scaler"]
orient = dist_pickle["orient"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
spatial_size = dist_pickle["spatial_size"]
hist_bins = dist_pickle["hist_bins"]



In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2

# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, test=False):
    
    draw_img = np.copy(img)
    if test==True:
        img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    
    #Change color space from RGB to YCrCb
    ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YCrCb)
    
    bboxes = []
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
    
    # Extract channels
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell)+1
    nyblocks = (ch1.shape[0] // pix_per_cell)+1 
    nfeat_per_block = orient*cell_per_block**2
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell)-1 
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    prefix=1
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)
           
            # Scale features and make a prediction
            combined_features = np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1)
            #print(combined_features.shape)
            test_features = X_scaler.transform(combined_features)    
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                
                boxLeftCorner = (xbox_left, ytop_draw+ystart)
                boxRightCorner = (xbox_left+win_draw,ytop_draw+win_draw+ystart)
                bboxes.append((boxLeftCorner,boxRightCorner ))
                #cv2.rectangle(draw_img, boxLeftCorner,boxRightCorner,(0,0,255),6)
            
    return  bboxes

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
from scipy.ndimage.measurements import label
# Add +1 heat to detected frames
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes

#Filter box detection with a threshold value
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

# Draw final boxes in the frame
def draw_labeled_bboxes(img, labels):
    bboxes_listi = []
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        bboxes_listi.append((bbox[0], bbox[1],))
        # Draw the box on the image
        if car_number == 3:
            color_box=(0,255,0)
        elif car_number == 2:
            color_box=(255,128,0)
        elif car_number == 1:
            color_box=(255,0,255)
        else:  
            color_box=(0, 0,255)
        cv2.rectangle(img, bbox[0], bbox[1], color_box, 6)
    # Return the image
    return img,bboxes_listi



In [ ]:
def deleteIsolatedSquares(bbox_intersect):
    
    # Create a list that will keep filtered boxes
    bbox_filtered= []

    for indx_boxA, boxA in enumerate(bbox_intersect):
        
        # Create a copy of the original list of boxes
        bbox_intersect_comparison = (bbox_intersect)[:]
        
        # Delete the element that is being compared from the copy
        # This will avoid to compare a box with itself
        del bbox_intersect_comparison[indx_boxA]
        
        # Enumerate bboxes B to be iterated
        bbox_intersect_comparison = enumerate(bbox_intersect_comparison)
        
        # Declare insersection flag
        intersect = True
        
        # Iterates through the list of boxes and check for intersection
        # with any of all other boxes in the list
        for indx_boxB, boxB in bbox_intersect_comparison:
            #Ax1 <= Bx2  &  Ax2 >= Bx1  & Ay1 <= By2  & Ay2 >= By1
            # (x1, y1) = (top_left_corner)
            # (x2, y2) = (bottom_right_corner)
            if (boxA[0][0] <= boxB[1][0] and boxA[1][0]  >= boxB[0][0] and
               boxA[0][1] <= boxB[1][1]  and boxA[1][1]  >= boxB[0][1] ):
                # If intersection is found, no need to continue with more comparison 
                intersect = True
                break
            else:
                # If there is no intersection, this variable will continue as False
                 intersect = False
        # If no intersection, does nothing; otherwise keeps the box and adds it to filtered list
        if intersect == False:
            print('delete ' + str(indx_boxA))
        else:
            bbox_filtered.append(boxA)
            
    # If the final list has zero element and the original had at least one
    # the original list is returned
    if len(bbox_filtered) == 0 and len(bbox_intersect) > 0:
        return  bbox_intersect
    else:
        return bbox_filtered
    

In [ ]:
# Pipeline that performs car detection with SVM 
def pipelineVideo(image):
    test= True
    global labels_previous, previous_boxes, iteration
    
    # Process frame for lane detection
    img =  laneLinesDetection.processLanePipeline(np.copy(image))
    #img = image
    #Create a zero mask for heat map
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    
    # Search with different window scales and in different regions according to window size
    
    # This was chosen to detec the car features when it is smaller (it' farther). 
    # IT is the most time consuming search since it creates more windows. 
    # IT is not neccesary to use this small window in the whole lower half. 
    ystart = 400
    ystop = 500
    scale = 1.2
    bboxes1 = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, test)
    
    # This second scale is chosen to detect average objects. 
    scale = 1.4
    ystart = 400
    ystop = 620
    bboxes2 = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, test)
    
    # This scale is the one that detects less object but also does not detect false positives
    # it helps to increase the whole area of detection once combined with previous scales. 
    scale = 1.8
    ystart = 400
    ystop = 656
    bboxes3 =  find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, test)
    
    # Remove isolated boxes wich most probably are false positives
    # the bigger scales are helped by smaller in order to avoid a true positive to be removed
    # Also the fisrt scale is helped by previous frame final detection (labels)
    bboxes1= deleteIsolatedSquares(bboxes1+labels_previous)
    bboxes2= deleteIsolatedSquares(bboxes2+ bboxes1)
    bboxes3= deleteIsolatedSquares(bboxes2 + bboxes3)
    
    # For debugging, prints if no object were detected 
    if len(bboxes3) <1:
        print('error: no squares')
    
    # Add current boxes to detections from previous frames
    previous_boxes.append( (bboxes3) )
    
    # Initiate final list of boxes
    bboxes_final = []
    # Add the boxes found in the previous frames to our current detection
    # also remove the last element in the list to keep at maximum of 4 frames 
    if len(previous_boxes)>4:
        previous_boxes.pop(0)
    for box in previous_boxes:
        bboxes_final += box
    
    # Add heat to each box in box list
    heat = add_heat(heat, bboxes_final)
    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,6)
    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    
    # Draw final boxes in the picture and get the list of drawn boxes
    draw_img, labels_previous = draw_labeled_bboxes(img, labels)
    
    # For debugging: prints when more than two objects are detected
    if (labels[1])>2:
        print("Foun: " + str(labels[1]) + " cars")
        
    
    return draw_img

In [ ]:
import matplotlib.image as mpimg
%matplotlib qt

from collections import deque
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import CarND_Advanced_Lane_Lines

# Instance Advanced Lane Detection Project
laneLinesDetection =  CarND_Advanced_Lane_Lines.AdvancedLaneFinding()
laneLinesDetection.prepareLaneDetection()

#Global variables for previous detections
labels_previous = []
previous_boxes = []

#Output video
white_output = './videos/project_video_lanes_vehicles.mp4'
# Input video
clip1 = VideoFileClip("./videos/project_video_lanes.mp4")

# Process the video frame by frame calling processImagePipeline
white_clip = clip1.fl_image(pipelineVideo) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)